<a href="https://colab.research.google.com/github/achmadgani/devops_training/blob/main/colletothricum_growth_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INTRODUCTION TO PYTHON

# Introduction

### Colletothricum Clades Implementation

In [ ]:
print("hello world")

hello world


# New Section

Code for Colletothricum Clades implementation python Program

In [6]:
import pandas as pd

class ColletotrichumGrowthCalculator:
    def __init__(self, csv_path):
        self.csv_path = csv_path
        self.df = pd.read_csv(csv_path)
        self.rename_columns()

    def rename_columns(self):
        # Rename columns by removing any parentheses and the content within them
        self.df.columns = [col.split('(')[0].strip() for col in self.df.columns]

    # def calculate_growth(self):
    #     # Add the calculation logic here
    #     self.df['growth'] = self.df.apply(lambda row: self.formula(row), axis=1)

    # def formula(self, row):

    def save_results(self, output_path):
        self.df.to_csv(output_path, index=False)

    def display_dataframe(self):
    # Use the built-in display function to nicely format the output in Jupyter/Colab
        display(self.df)

# Usage:
# Initialize the calculator with the path to your CSV file
calculator = ColletotrichumGrowthCalculator('/content/mulberry - tsukuba data.csv')
# calculator.calculate_growth()
calculator.display_dataframe()
# calculator.save_results('path_to_output_csv.csv')


,datetime,temperature,precipitation,humidity,wetness_duration
0,1993/7/1 1:00,16.5,0.0,81,0
1,1993/7/1 2:00,16.9,0.0,79,0
2,1993/7/1 3:00,16.6,0.0,80,0
3,1993/7/1 4:00,15.1,0.0,87,0
4,1993/7/1 5:00,15.8,0.0,85,0
...,...,...,...,...,...
3643,1993/11/29 20:00,20.2,0.0,92,8
3644,1993/11/29 21:00,20.4,0.0,91,8
3645,1993/11/29 22:00,20.7,0.0,92,8
3646,1993/11/29 23:00,20.7,0.0,92,8
